# NSQIP - Stepwise Regression
---

Regressing on all of our predictors at once will tend to produce an over-fitted model. A traditional approach to this is to use a stepwise selection procedure, fitting individual best-fitting terms in sequence and adding them to build up a model in an interative fashion. These models are pretty common in the literature, but statisticians tend not to like them for a variety of reasons. Some of these are technical, but a pretty basic reason is that doing dozens of repeated statistical tests makes the meaning of each statistical test less valid.

Regardless, we'll fit a stepwise logistic regression model here. To be clear, our model is of the form: 

$$P(y|x) = \frac{1}{1 + e^{-(\beta_0 + \sum_{i}\beta_i x_i)}} $$

... where $x$ is a vector of predictors, and $\beta$ is a vector of coefficients. For variable selection at each step, we'll take the new model (produced by either adding a variable or dropping one) that provides the biggest reduction of the [Akaike Information Criterion](https://en.wikipedia.org/wiki/Akaike_information_criterion). We stop making changes when no addition or removal of a variable reduces the AIC.

We'll try to apply some of the modern bootstrapping technology here to define confidence intervals and estimate model performance. Our strategy will be
 - Divide the data into training and testing sets.
 - Fit a Stepwise Regression to the training set.
 - Use a bootstrap procedure to determine confidence intervals for the coefficients.
 - Check performance on the testing set.

Begin with imports of key libraries, and load the data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer
from stepwiseLogisticRegression import stepwiseLogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV
import sklearn.metrics as sklm
import random
import warnings 
import math
warnings.filterwarnings("ignore") # Ignore annoying warnings

dataDir = './Data/'
mungedFileName = dataDir + 'mungedData.pkl'

cdf = pd.read_pickle(mungedFileName)

Next, determine which response and predictor columns we want to use.

In [2]:
# y is True for bad things happening.
# 'readmission', 'reoperation', 'morbidity', 'mortality', 'infection'
from outcomeVariables import getOutcomeVariable
yFull = getOutcomeVariable(cdf, 'morbidity')

# Drop rows with NaN outcome y data
nanIdx = np.isnan(yFull).nonzero()
yFull = np.delete(yFull.ravel(), nanIdx ,axis=0).reshape(-1,1)
cdf.drop(cdf.index[nanIdx], axis=0, inplace=True)

# Add a BMI column
cdf.loc[:,'BMI'] = cdf.loc[:,'WEIGHT']*(1/2.20462)/(cdf.loc[:,'HEIGHT']*(1/39.3701)*\
                                                    cdf.loc[:,'HEIGHT']*(1/39.3701))

# List of predictors to keep
keepList1 = ['SEX', 'RACE_NEW','ETHNICITY_HISPANIC','AGE','ANESTHES','HEIGHT','WEIGHT','BMI',\
             'DIABETES','SMOKE','PACKS','ETOH','DYSPNEA','FNSTATUS2','VENTILAT',\
             'HXCOPD','CPNEUMON','HXCHF','HXMI','PRVPCI','PRVPCS','HXANGINA','HYPERMED',\
             'HXPVD','RENAFAIL','DIALYSIS','CVA','DISCANCR','WNDINF-','STEROID','WTLOSS',\
             'BLEEDDIS','PROPER30','ASACLASS','FNSTATUS1','RBC']
# An additional list of predictors that might be interesting
keepList2 = ['ASCITES','PRSODM','PRBUN','PRCREAT','PRALBUM','PRBILI','PRSGOT','PRALKPH',\
             'PRWBC','PRHCT','PRPLATE','PRPTT','PRINR','PRPT','PGY','ESOVAR','RESTPAIN',\
             'HXTIA','TRANSFUS','CHEMO','RADIO','PRSEPSIS','PREGNANCY','EMERGNCY',\
             'OPTIME','MALLAMP']

# Combine the lists
keepList = keepList1 + keepList2
# Make a list of any column in cdf whose name starts with a string in keepList
colsToKeep = [colName for colName in cdf.columns \
              if np.any([colName.startswith(keepItem) for keepItem in keepList])]
cdf = cdf[colsToKeep]

We will drop predictors that don't have any variance. (This can happen because we already dropped a bunch of cases for which our outcomes are not recorded.)

These regression models will fail on missing data, so we'll impute missing data for each column by setting it equal to the mean.

We need to add an explicit intercept term.

There's no regularization in this model, so there's no need to scale coefficients. This has the benefit of leaving us with interpretable coefficients.

In [3]:
# Protect against only having one value in the columns.
# (Imputer will drop these columns, and then we won't know the names...)
for colName in cdf.columns:
    colData = cdf[colName]
    uniques = np.unique(colData)
    nonNanUniques = [item for item in uniques if ~np.isnan(item)]
    if len(nonNanUniques) < 2:
        print('Not enough variance, dropping column: %s' % colName)
        cdf.drop(colName, axis=1, inplace=True)

# Impute missing data in cdf
colNames = cdf.columns
imp = Imputer(missing_values='NaN', strategy='mean', axis=0, verbose=True)
imp.fit(cdf)
X = imp.transform(cdf)
X = pd.DataFrame(X, columns=colNames)

# Add an intercept term.
X['intercept'] = 1.0

# Scale the columns
# X = StandardScaler().fit_transform(cdf)
# scaledX = pd.DataFrame(X, columns=colNames)
scaledX = X

We break the data into separate fractions for training (2/3) and testing (1/3). All of our fitting and cross-validation will be done on the training set. The testing set will be held out to evaluate the final model performance.

In [ ]:
# Split for validation
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaledX, yFull, test_size=0.33)
y_train = y_train.reshape(-1,)
y_test = y_test.reshape(-1,)

We'll create a stepwise regression object. This is unregularized. If we try to add a predictor to the model that's not linearly independent of other predictors, the regression will fail because the covariance matrix is singular. We just discard these as potential solutions.

In [ ]:
# Do stepwise logistic regression, based on AIC
swl = stepwiseLogisticRegression()

# Bootstrap this whole regression
nBoots = 2
nSamp = X_train.shape[0]
nPred = X_train.shape[1]
bootParams = np.empty((nBoots, nPred))
bootNParams = np.empty((nBoots,1))
# For each bootstrap sample
for bootN in np.arange(nBoots):
    # Draw a collection of observations from the training set with replacement
    sampIdx = [random.randint(0,nSamp-1) for n in np.arange(nSamp)]
    y_boot = y_train[sampIdx].reshape(-1,)
    X_boot = X_train.iloc[sampIdx,:]
    X_boot = pd.DataFrame(X_boot, columns=X.columns)
    
    # No need to scale the columns
    # X_boot = StandardScaler().fit_transform(X_boot)
    # X_boot = pd.DataFrame(X_boot, columns=colNames)
    
    # Fit the model to the bootstrap sample. 
    swl = swl.fit(X_boot, y_boot, disp=False)
    
    # Store the coefficients (and the # of predictors) for each bootstrapped sample
    coeffList = pd.Series(0.0,index=X.columns)
    for index, colName in enumerate(swl.useCols):
        coeffList[colName] = swl.model.coef_[0,index]
    bootParams[bootN,:] = coeffList.values
    bootNParams[bootN] = sum(coeffList.values != 0)
    print('Booted #%d/%d with Nparams: %.3f' % (bootN+1, nBoots, bootNParams[bootN]))

baseAIC = 4726.272
	 Adding: AIC = 4435.22 	 PRHCT
baseAIC = 4435.221
	 Adding: AIC = 4315.96 	 WNDINF-Yes
baseAIC = 4315.964
	 Adding: AIC = 4234.62 	 AGE
baseAIC = 4234.622
	 Adding: AIC = 4172.28 	 VENTILAT-Yes
baseAIC = 4172.277
	 Adding: AIC = 4134.19 	 DIABETES-INSULIN
baseAIC = 4134.189
	 Adding: AIC = 4100.03 	 OPTIME
baseAIC = 4100.073
	 Adding: AIC = 4075.31 	 BLEEDDIS-Yes
baseAIC = 4075.312
	 Adding: AIC = 4062.35 	 PRPTT
baseAIC = 4062.352
	 Adding: AIC = 4050.06 	 HXCOPD-Yes
baseAIC = 4050.266
	 Adding: AIC = 4038.48 	 PRPLATE
baseAIC = 4038.480
	 Adding: AIC = 4024.02 	 PRWBC
baseAIC = 4024.034
	 Adding: AIC = 4015.79 	 PRVPCI-Yes
baseAIC = 4015.822
	 Adding: AIC = 4009.65 	 PRALBUM
baseAIC = 4008.076
	 Adding: AIC = 4002.65 	 CVA-Yes
baseAIC = 4004.581
	 Adding: AIC = 3997.52 	 HYPERMED-Yes
baseAIC = 3999.191
	 Adding: AIC = 3993.64 	 PRALKPH
baseAIC = 3994.852
	 Adding: AIC = 3989.31 	 SMOKE-Yes
baseAIC = 3992.318
	 Adding: AIC = 3984.42 	 FNSTATUS1-Totally Dependent
ba

In [ ]:
# Now that we've bootstrapped, we can fit the model to the actual training data sample
bestEst = swl.fit(X_train, y_train, disp=False)
coeffList = pd.Series(0.0,index=X.columns)
for index, colName in enumerate(bestEst.useCols):
        coeffList[colName] = bestEst.model.coef_[0,index]
coeffs = coeffList.values
nParams = sum(coeffList.values != 0)
print('N params: %d' % nParams)

In [ ]:
# Identify confidence intervals from the parameters fit to the bootstrapped samples
lciIdx = round(.025*(nBoots-1))
uciIdx = round(.975*(nBoots-1))
bootParams.shape
bootParams.sort(axis=0)
lowCI = bootParams[lciIdx,:]
upCI  = bootParams[uciIdx,:]
allCols = [(coeffs[index], lowCI[index], upCI[index], colName) for index, colName\
           in enumerate(X_boot.columns)]

Now print out significant coefficients. 

In [ ]:
# Print out the coefficient if it was included in the final model
# ... or if it was significant in the bootstrap.
print('--- Terms in final model, and significant ---')
for item in allCols:
    if (item[0] != 0) and ((item[1] > 0) or (item[2] < 0)):
        print('Coeff: %+.3f [%+.3f, %+.3f] %s  ' % item)
print('--- Terms in final model, and NOT significant ---')
for item in allCols:
    if (item[0] != 0) and ~((item[1] > 0) or (item[2] < 0)):
        print('Coeff: %+.3f [%+.3f, %+.3f] %s  ' % item)
print('--- Terms NOT in final model, and significant ---')
for item in allCols:
    if (item[0] == 0) and ((item[1] > 0) or (item[2] < 0)):
        print('Coeff: %+.3f [%+.3f, %+.3f] %s  ' % item)
print('--- Terms NOT in final model, and NOT significant ---')
for item in allCols:
    if (item[0] == 0) and ~((item[1] > 0) or (item[2] < 0)):
        print('Coeff: %+.3f [%+.3f, %+.3f] %s  ' % item)

Plot ROC curves for in-sample training data, and test data. The test data here was not used to fit the model at all, so it should approximate how our model will perform prospectively on new data. Furthermore, if we've done a good job regularizing, the in-sample performance should be pretty close to the test data performance.

In [ ]:
# Plot an ROC curve for training
plt.clf
pred_train = swl.predict_proba(X_train)
rocAUC_train = sklm.roc_auc_score(y_train,pred_train)
print('In-sample ROC AUC = %.3f ' % rocAUC_train)
fpr, tpr, _ = sklm.roc_curve(y_train, pred_train)
plt.figure()
plt.plot(fpr,tpr,label='In-sample')

# Plot an ROC curve for test data
pred_test = swl.predict_proba(X_test)
rocAUC_test = sklm.roc_auc_score(y_test,pred_test)
print('Test ROC AUC = %.3f ' % rocAUC_test)
fpr, tpr, _ = sklm.roc_curve(y_test, pred_test)
plt.plot(fpr,tpr,label='Test')

# Format the ROC plot
plt.plot([0, 1], [0, 1], color='k', linestyle=':')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.gca().set_aspect('equal', adjustable='box')
plt.legend(loc='lower right')
plt.show()

In this case you can see here how poorly this model generalizes to new test data.